# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('stopwords')
nltk.download(['punkt', 'wordnet'])
from nltk.corpus import stopwords
import re
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
import pickle

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///myProjectDatabase.db')
df = pd.read_sql_table('Categories_Table', engine)  

#display(Y.head())
#print(Y.shape)

In [3]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
df.describe()

,id,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
count,26216.00000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,...,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000
mean,15224.82133,0.773650,0.170659,0.004501,0.414251,0.079493,0.050084,0.027617,0.017966,0.032804,...,0.011787,0.043904,0.278341,0.082202,0.093187,0.010757,0.093645,0.020217,0.052487,0.193584
std,8826.88914,0.435276,0.376218,0.066940,0.492602,0.270513,0.218122,0.163875,0.132831,0.178128,...,0.107927,0.204887,0.448191,0.274677,0.290700,0.103158,0.291340,0.140743,0.223011,0.395114
min,2.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7446.75000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,15662.50000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,22924.25000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,30265.00000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [5]:
X = df['message']
y = df[df.columns.difference(['id', 'message','original', 'genre'])]

In [6]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [7]:
type(X)

pandas.core.series.Series

In [8]:
y.head()

,aid_centers,aid_related,buildings,child_alone,clothing,cold,death,direct_report,earthquake,electricity,...,request,search_and_rescue,security,shelter,shops,storm,tools,transport,water,weather_related
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
type(y)

pandas.core.frame.DataFrame

### 2. Write a tokenization function to process your text data

In [10]:
def tokenize(text):
    text = re.sub(r"[^a-z0-9]", " ", text.lower())

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).strip()
        if clean_tok not in stopwords.words("english"):
            clean_tokens.append(clean_tok)

    return clean_tokens

In [11]:
print(tokenize('Weather update - a cold front from Cuba that could pass over Haiti'))

['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [12]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)), 
        ('tfidf',TfidfTransformer()),
        ('clf',MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [13]:
#Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Train pipeline
fitted_pipeline = pipeline.fit(X_train, y_train)

In [14]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(19662,)
(6554,)
(19662, 36)
(6554, 36)


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [15]:
 # predict on test data
y_pred_np = fitted_pipeline.predict(X_test)

In [16]:
#converting to dataframe
y_pred = pd.DataFrame (y_pred_np, columns = y_test.columns)
display(y_test.head())
display(y_pred.head())

,aid_centers,aid_related,buildings,child_alone,clothing,cold,death,direct_report,earthquake,electricity,...,request,search_and_rescue,security,shelter,shops,storm,tools,transport,water,weather_related
14905,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
4076,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
10765,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,1
4943,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17445,0,1,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,1,0,1


,aid_centers,aid_related,buildings,child_alone,clothing,cold,death,direct_report,earthquake,electricity,...,request,search_and_rescue,security,shelter,shops,storm,tools,transport,water,weather_related
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [17]:
for col in y_test.columns:
    #returning dictionary from classification report
    #print(col)
    #display(y_test[col].head())
    print("Category:", col)
    class_col= classification_report(y_true = y_test.loc [:,col], y_pred = y_pred.loc [:,col])
    print(class_col)


             precision    recall  f1-score   support

          0       0.99      1.00      0.99      6481
          1       0.00      0.00      0.00        73

avg / total       0.98      0.99      0.98      6554

             precision    recall  f1-score   support

          0       0.74      0.87      0.80      3848
          1       0.75      0.57      0.65      2706

avg / total       0.75      0.75      0.74      6554

             precision    recall  f1-score   support

          0       0.95      1.00      0.98      6204
          1       0.83      0.13      0.22       350

avg / total       0.95      0.95      0.93      6554

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6554

avg / total       1.00      1.00      1.00      6554

             precision    recall  f1-score   support

          0       0.99      1.00      0.99      6470
          1       0.77      0.12      0.21        84

avg / total       0.99      0.99

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [18]:
#y_true = [0, 1, 2, 2, 2]
#y_pred = [0, 0, 2, 2, 1]
#category_names = y.columns.values
##print(len(category_names))
#for i in range(len(category_names)):
    #target_names = ['{col_name}_0'.format(col_name=category_names[i]), '{col_name}_1'.format(col_name=category_names[i])]
    #print("Category Name:", category_names[i])
    #print(classification_report(y_test[:,i], y_pred[:,i]))#, target_names = target_names))

#print(classification_report(y_test.values, y_pred, target_names=category_names))

#print(classification_report(y_test.values, y_pred, target_names=y.columns.values))

### 6. Improve your model
Use grid search to find better parameters. 

In [19]:
# specify parameters for grid search
parameters = {
        'vect__analyzer': ['word'],
        'vect__max_features': [5, 50],
        'clf__estimator__n_estimators': [10, 20],
        'clf__estimator__min_samples_split': [2, 5]
    }

# create grid search object
cv = GridSearchCV(pipeline, param_grid=parameters)

In [20]:
cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'vect__analyzer': ['word'], 'vect__max_features': [5, 50], 'clf__estimator__n_estimators': [10, 20], 'clf__estimator__min_samples_split': [2, 5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [21]:
# predict on test data
y_pred_grid = cv.predict(X_test)

In [24]:
#converting to dataframe
y_pred_grid_df = pd.DataFrame (y_pred_grid, columns = y_test.columns)
display(y_test.head())
display(y_pred_grid_df.head())

,aid_centers,aid_related,buildings,child_alone,clothing,cold,death,direct_report,earthquake,electricity,...,request,search_and_rescue,security,shelter,shops,storm,tools,transport,water,weather_related
14905,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
4076,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
10765,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,1
4943,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17445,0,1,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,1,0,1


,aid_centers,aid_related,buildings,child_alone,clothing,cold,death,direct_report,earthquake,electricity,...,request,search_and_rescue,security,shelter,shops,storm,tools,transport,water,weather_related
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
for col in y_test.columns:
    #returning dictionary from classification report
    #print(col)
    #display(y_test[col].head())
    class_col= classification_report(y_true = y_test.loc [:,col], y_pred = y_pred_grid_df.loc [:,col])
    print(class_col)

             precision    recall  f1-score   support

          0       0.99      1.00      0.99      6481
          1       0.00      0.00      0.00        73

avg / total       0.98      0.99      0.98      6554

             precision    recall  f1-score   support

          0       0.72      0.82      0.77      3848
          1       0.69      0.55      0.61      2706

avg / total       0.71      0.71      0.71      6554

             precision    recall  f1-score   support

          0       0.95      0.99      0.97      6204
          1       0.45      0.13      0.20       350

avg / total       0.93      0.95      0.93      6554

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6554

avg / total       1.00      1.00      1.00      6554

             precision    recall  f1-score   support

          0       0.99      1.00      0.99      6470
          1       0.00      0.00      0.00        84

avg / total       0.97      0.99

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [26]:
# Save Model Using Pickle
# save the model to disk
filename = 'finalized_model.sav'
pickle.dump(cv, open(filename, 'wb'))
 
# To Use later...
# load the model from disk
#loaded_model = pickle.load(open(filename, 'rb'))
#result = loaded_model.score(X_test, y_test)
#print(result)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.